In [20]:
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from requests.exceptions import ConnectionError
from config import api_key, api_key2, api_key3, api_key4
#second key for when we run into too many requests issue

In [18]:
title_basics = pd.read_csv("./Resources/title.basics.tsv/data.tsv", sep = "\t", lineterminator = "\n")
#use title_basics, join on imdb tconst and omdb imdbID, adds isAdult, originalTitle

C:\Users\snigd\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
title_basics.head()
title_basics.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult            int64
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [4]:
#Get the exact string for all null values, in case we need to delete anything
nullValue = (title_basics['endYear'].iloc[0])
nullValue

'\\N'

In [19]:
#Get rid of columns we don't need and rename to make pretty and readable
title_basics = title_basics[['tconst','primaryTitle','originalTitle','isAdult','startYear','endYear']].rename(columns = {'tconst':'imdbID','primaryTitle': 'Primary Title','originalTitle' : 'Orig. Title','isAdult': 'Adult Film','startYear': 'Start Year','endYear': 'End Year'})

#The only things we need to be not null are titles, imdb ID
#None of them seem to be null but just in case let's make a condition
#get rid of tv episodes, we are only looking at films and other
title_basics = title_basics[title_basics['Primary Title'] != nullValue]
title_basics = title_basics[title_basics['Orig. Title'] != nullValue]

#Now we have a cleaned imdb set, from which we will pull Primary + OG titles
#Adult Film, and Start/End year. 
title_basics.head()

,imdbID,Primary Title,Orig. Title,Adult Film,Start Year,End Year
0,tt0000001,Carmencita,Carmencita,0,1894,\N
1,tt0000002,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N
2,tt0000003,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N
3,tt0000004,Un bon bock,Un bon bock,0,1892,\N
4,tt0000005,Blacksmith Scene,Blacksmith Scene,0,1893,\N


In [21]:
#Now choose a random 1000 rows from this DataFrame, since the free api of omdb only allows 1000 queries per day
title_basics = title_basics.sample(n=1000)
#len(title_basics['filmID'])
title_basics = title_basics.reset_index(drop=True)
title_basics.head()

,imdbID,Primary Title,Orig. Title,Adult Film,Start Year,End Year
0,tt1056832,Episode #1.669,Episode #1.669,0,1967,\N
1,tt0893986,122B,122B,0,1977,\N
2,tt2682868,Episode dated 27 March 1972,Episode dated 27 March 1972,0,1972,\N
3,tt0873091,Bargeld,Bargeld,0,1986,\N
4,tt5642870,Oscar-Nominated Documentary 'The Invisible War',Oscar-Nominated Documentary 'The Invisible War',0,2013,\N


In [22]:
#create a test dictionary
imdbID = title_basics['imdbID'].iloc[[0]]
#print(imdbID.values[0])
url =  f"http://www.omdbapi.com/?i={imdbID.values[0]}&apikey={api_key4}"
data = requests.get(url)
datajson = data.json()
datajson

{'Title': 'Episode #1.669',
 'Year': '1967',
 'Rated': 'N/A',
 'Released': 'N/A',
 'Season': 'N/A',
 'Episode': 'N/A',
 'Runtime': 'N/A',
 'Genre': 'Drama',
 'Director': 'Michael Apted',
 'Writer': 'Jim Allen',
 'Actors': 'Peter Adamson, Jean Alexander, Paul Bacon, James Beck',
 'Plot': 'N/A',
 'Language': 'N/A',
 'Country': 'N/A',
 'Awards': 'N/A',
 'Poster': 'http://ia.media-imdb.com/images/M/MV5BMTIyODk5MzgyMl5BMl5BanBnXkFtZTYwNDY0Njc5._V1_SX300.jpg',
 'Ratings': [],
 'Metascore': 'N/A',
 'imdbRating': 'N/A',
 'imdbVotes': 'N/A',
 'imdbID': 'tt1056832',
 'seriesID': 'N/A',
 'Type': 'episode',
 'Response': 'True'}

In [23]:
#create an empty dictionary which is what we will use to append all
#omdb data
#We are ignoring things like season and episode as we're focusing on the 
#big picture info mainly regarding films
datadict = {
     'imdbID': [],
     'Year': [],
     'Genre': [],
     'Rated': [],
     'Released': [],
     'Runtime': [], 
     'Director': [],
     'Writer': [],
     'Actors': [],
     'Plot': [],
     'Language': [],
     'Country': [],
     'Awards': [],
     'Ratings': [],
     'imdbRating': [],
     'Type': []}


In [24]:
#loop through all the film IDs in the imdb dataset
filmID = title_basics['imdbID']
tempDict = {}

for idx, val in enumerate(filmID):
    url = f"http://www.omdbapi.com/?i={val}&apikey={api_key4}"
    tempdata = requests.get(url)
    tempDict = tempdata.json()
    if tempdata.status_code == 404 or tempDict['Response']=='False':
        continue
    
    #pprint(tempDict)
    datadict['imdbID'].append(tempDict['imdbID'])
    datadict['Genre'].append(tempDict['Genre'])
    datadict['Year'].append(tempDict['Year'])
    datadict['Rated'].append(tempDict['Rated'])
    datadict['Released'].append(tempDict['Released'])
    datadict['Runtime'].append(tempDict['Runtime'])
    datadict['Director'].append(tempDict['Director'])
    datadict['Writer'].append(tempDict['Writer'])
    datadict['Actors'].append(tempDict['Actors'])
    datadict['Plot'].append(tempDict['Plot'])    
    datadict['Language'].append(tempDict['Language'])
    datadict['Country'].append(tempDict['Country'])
    datadict['Awards'].append(tempDict['Awards'])
    datadict['Ratings'].append(tempDict['Ratings'])
    datadict['imdbRating'].append(tempDict['imdbRating'])
    datadict['Type'].append(tempDict['Type'])

In [25]:
omdbdf = pd.DataFrame(datadict)
omdbdf.head()

,imdbID,Year,Genre,Rated,Released,Runtime,Director,Writer,Actors,Plot,Language,Country,Awards,Ratings,imdbRating,Type
0,tt1056832,1967,Drama,N/A,N/A,N/A,Michael Apted,Jim Allen,"Peter Adamson, Jean Alexander, Paul Bacon, Jam...",N/A,N/A,N/A,N/A,[],N/A,episode
1,tt0893986,1977,"Family, Comedy",TV-G,05 Apr 1977,28 min,Bob Schwarz,N/A,"Luis Avalos, Jim Boyd, Morgan Freeman, Judy Gr...",N/A,English,USA,N/A,[],N/A,episode
2,tt2682868,1972,News,N/A,27 Mar 1972,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,[],N/A,episode
3,tt0873091,1986,Drama,N/A,26 Jan 1986,30 min,Hans W. Geissendörfer,"Hans W. Geissendörfer, Barbara Piazza","Domna Adamopoulou, Selma Baldursson, Johanna B...",N/A,N/A,N/A,N/A,"[{'Source': 'Internet Movie Database', 'Value'...",6.4,episode
4,tt5642870,2013,News,N/A,05 Feb 2013,N/A,N/A,N/A,"Judith Broder, Ellen DaSilva, Kirby Dick, Jaco...",N/A,N/A,N/A,N/A,[],N/A,episode


In [47]:
title_basics.head()

,index,imdbID,Primary Title,Orig. Title,Adult Film,Start Year,End Year
0,0,tt1056832,Episode #1.669,Episode #1.669,0,1967,\N
1,1,tt0893986,122B,122B,0,1977,\N
2,2,tt2682868,Episode dated 27 March 1972,Episode dated 27 March 1972,0,1972,\N
3,3,tt0873091,Bargeld,Bargeld,0,1986,\N
4,4,tt5642870,Oscar-Nominated Documentary 'The Invisible War',Oscar-Nominated Documentary 'The Invisible War',0,2013,\N


In [26]:
oID = omdbdf['imdbID']
iID = title_basics['imdbID']
ocount = 0

#go through imdb df and delete rows where we couldn't find the film/show
for row in range(len(iID)):
    if iID[row]!=oID[ocount]:
        title_basics = title_basics.drop([row])
        continue
    else:
        ocount+=1

In [27]:
title_basics = title_basics.reset_index(drop=True)
title_basics.head()

,imdbID,Primary Title,Orig. Title,Adult Film,Start Year,End Year
0,tt1056832,Episode #1.669,Episode #1.669,0,1967,\N
1,tt0893986,122B,122B,0,1977,\N
2,tt2682868,Episode dated 27 March 1972,Episode dated 27 March 1972,0,1972,\N
3,tt0873091,Bargeld,Bargeld,0,1986,\N
4,tt5642870,Oscar-Nominated Documentary 'The Invisible War',Oscar-Nominated Documentary 'The Invisible War',0,2013,\N


In [35]:
omdbdf = omdbdf.rename(columns={'imdbID':'filmID'})
omdbdf.head(20)

,filmID,Year,Genre,Rated,Released,Runtime,Director,Writer,Actors,Plot,Language,Country,Awards,Ratings,imdbRating,Type
0,tt1056832,1967,Drama,N/A,N/A,N/A,Michael Apted,Jim Allen,"Peter Adamson, Jean Alexander, Paul Bacon, Jam...",N/A,N/A,N/A,N/A,[],N/A,episode
1,tt0893986,1977,"Family, Comedy",TV-G,05 Apr 1977,28 min,Bob Schwarz,N/A,"Luis Avalos, Jim Boyd, Morgan Freeman, Judy Gr...",N/A,English,USA,N/A,[],N/A,episode
2,tt2682868,1972,News,N/A,27 Mar 1972,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,[],N/A,episode
3,tt0873091,1986,Drama,N/A,26 Jan 1986,30 min,Hans W. Geissendörfer,"Hans W. Geissendörfer, Barbara Piazza","Domna Adamopoulou, Selma Baldursson, Johanna B...",N/A,N/A,N/A,N/A,"[{'Source': 'Internet Movie Database', 'Value'...",6.4,episode
4,tt5642870,2013,News,N/A,05 Feb 2013,N/A,N/A,N/A,"Judith Broder, Ellen DaSilva, Kirby Dick, Jaco...",N/A,N/A,N/A,N/A,[],N/A,episode
5,tt6497354,2014,Talk-Show,N/A,02 Aug 2014,N/A,"Kevin C. Kellum, Stephanie Kellum",Stephanie Kellum,N/A,N/A,N/A,N/A,N/A,[],N/A,episode
6,tt0623830,2000,"Crime, Drama",N/A,09 Feb 2000,45 min,Carl Lang,"Silke Holtheide, Petra Lüschow, Raimund Maessen","Rüdiger Joswig, Ursula Buschhorn, Rainer Based...",N/A,N/A,N/A,N/A,[],N/A,episode
7,tt3658810,1907,Short,N/A,N/A,N/A,Luigi Maggi,N/A,N/A,N/A,Italian,Italy,N/A,[],N/A,movie
8,tt5554840,2016,Comedy,N/A,28 Mar 2016,25 min,N/A,N/A,"Anna Svava Knútsdóttir, Vignir Rafn Valþórsson...",N/A,Icelandic,Iceland,N/A,[],N/A,series
9,tt2197031,2011,Reality-TV,N/A,08 Jan 2011,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,[],N/A,episode


In [46]:
#now we can concatenate them. Have to concat because for some reason astype isn't working or anything else, so can't join on 'object' columns
joined = pd.concat([title_basics, omdbdf], axis=1, join_axes=[title_basics.index])

#getting rid of redundant columns as well as End Year since most of End Year was N/A and it's not really a point of interest
joined = joined.drop(['filmID','Year','index', 'End Year'],axis=1).rename(columns={'Adult Film':'Adult'}).set_index('imdbID')
joined.head()

,Primary Title,Orig. Title,Adult,Start Year,Genre,Rated,Released,Runtime,Director,Writer,Actors,Plot,Language,Country,Awards,Ratings,imdbRating,Type
imdbID,,,,,,,,,,,,,,,,,,
tt1056832,Episode #1.669,Episode #1.669,0,1967,Drama,N/A,N/A,N/A,Michael Apted,Jim Allen,"Peter Adamson, Jean Alexander, Paul Bacon, Jam...",N/A,N/A,N/A,N/A,[],N/A,episode
tt0893986,122B,122B,0,1977,"Family, Comedy",TV-G,05 Apr 1977,28 min,Bob Schwarz,N/A,"Luis Avalos, Jim Boyd, Morgan Freeman, Judy Gr...",N/A,English,USA,N/A,[],N/A,episode
tt2682868,Episode dated 27 March 1972,Episode dated 27 March 1972,0,1972,News,N/A,27 Mar 1972,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,[],N/A,episode
tt0873091,Bargeld,Bargeld,0,1986,Drama,N/A,26 Jan 1986,30 min,Hans W. Geissendörfer,"Hans W. Geissendörfer, Barbara Piazza","Domna Adamopoulou, Selma Baldursson, Johanna B...",N/A,N/A,N/A,N/A,"[{'Source': 'Internet Movie Database', 'Value'...",6.4,episode
tt5642870,Oscar-Nominated Documentary 'The Invisible War',Oscar-Nominated Documentary 'The Invisible War',0,2013,News,N/A,05 Feb 2013,N/A,N/A,N/A,"Judith Broder, Ellen DaSilva, Kirby Dick, Jaco...",N/A,N/A,N/A,N/A,[],N/A,episode


In [48]:
joined.to_csv('cleanedData.csv')